In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd

import japanize_matplotlib
import pybaseball
from pybaseball import statcast

pybaseball.cache.enable()
data = statcast(start_dt = '2008-04-01', end_dt='2008-09-30')
data[data['player_name'].str.contains('Johnson, Randy')]

In [ ]:
data_randy = data[data['player_name']=='Johnson, Randy']
data_randy.to_csv('2008_Randy.csv')
data_randy.head()

In [ ]:
df = pd.read_csv('2008_Randy.csv')
df

In [ ]:
counts = df['pitch_type'].value_counts()
plt.pie(counts, labels=counts.index, counterclock=False, startangle=90)
plt.title('Randyの球種割合')
plt.show()

In [ ]:
print(df['pitch_type'].unique())

In [ ]:
print(df['pitch_type'].value_counts())

In [ ]:
print(df['pitch_type'].isnull().sum())

In [ ]:
df['speed_km'] = df['release_speed'] * 1.61
df.head()

In [ ]:
print(df['speed_km'].max(), df['speed_km'].min())

In [ ]:
Fastball = df[df['pitch_type']=='FF'] #ストレート
Slider = df[df['pitch_type']=='SL'] #スライダー
Sinker = df[df['pitch_type']=='SI'] #シンカー
Splitter = df[df['pitch_type']=='FS'] #スプリット

x = ['ストレート', 'スライダー', 'シンカー', 'スプリット']
y = [Fastball['speed_km'].mean(),
     Slider['speed_km'].mean(),
     Sinker['speed_km'].mean(),
     Splitter['speed_km'].mean()]

plt.bar(x, y)
plt.title('球種ごとの平均球速')
plt.ylim(110, 160)
plt.show()

In [ ]:
x, y = [], []

for i in range(1, 10):
    inning = Fastball[Fastball['inning']==i]
    x.append(i)
    y.append(inning['speed_km'].mean())

y

In [ ]:
plt.plot(x, y)
plt.title('イニングごとのストレート平均球速')
plt.xlabel('イニング（回）')
plt.ylabel('ストレートの平均球速（km/h）')
plt.show()

In [ ]:
border = [110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165]

Fastball = df[df['pitch_type']=='FF'] #ストレート
Slider = df[df['pitch_type']=='SL'] #スライダー
Sinker = df[df['pitch_type']=='SI'] #シンカー
Splitter = df[df['pitch_type']=='FS'] #スプリット

plt.hist(Fastball['speed_km'], alpha=0.7, hatch='/', label='ストレート', bins=border)
plt.hist(Slider['speed_km'], alpha=0.7, hatch='o', label='スライダー', bins=border)
plt.hist(Sinker['speed_km'], alpha=0.7, hatch='.', label='シンカー', bins=border)
plt.hist(Splitter['speed_km'], alpha=0.7, hatch='*', label='スプリット', bins=border)

plt.title('球種ごとの球速分布')
plt.xlabel('球速（km/h）')
plt.ylabel('投球数')
plt.legend(loc='upper left')
plt.show()

In [ ]:
df['plate_x_cm'] = df['plate_x'] * 30.48
df['plate_z_cm'] = df['plate_z'] * 30.48
df.head()

In [ ]:
Fastball = df[df['pitch_type']=='FF']
Fastball.head()

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(Fastball['plate_x_cm'], Fastball['plate_z_cm'])

plt.title('ストレートの投球位置（捕手目線）')
plt.xlabel('投球のコース（cm）')
plt.ylabel('投球の高さ（cm）')
plt.xlim(-80, 80)
plt.ylim(0, 150)
plt.hlines(y=[50, 110], xmin=-30, xmax=30, color='black')
plt.vlines(x=[-30, 30], ymin=50, ymax=110, color='black')
plt.show()

In [ ]:
Swinging = Fastball[Fastball['description'].isin(['swinging_strike', 'swinging_strike_blocked'])]
Called = Fastball[Fastball['description']=='called_strike']

plt.figure(figsize=(10,10))
plt.scatter(Swinging['plate_x_cm'], Swinging['plate_z_cm'], color='red', label='空振り')
plt.scatter(Called['plate_x_cm'], Called['plate_z_cm'], color='blue', label='見逃し')

plt.title('ストレートの投球位置')
plt.xlabel('投球のコース（cm）')
plt.ylabel('投球の高さ（cm）')
plt.legend(loc='upper right')
plt.hlines(y=[50, 110], xmin=-30, xmax=30, color='black')
plt.vlines(x=[-30, 30], ymin=50, ymax=110, color='black')
plt.xlim(-80, 80)
plt.ylim(0, 150)
plt.show()

In [ ]:
location = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
total = 0
for height, side in zip(Fastball['plate_z_cm'], Fastball['plate_x_cm']):
    
    if height < 50 or 110 < height:
        continue
    if side < -30 or 30 < side:
        continue
    
    if 90 < height <= 110:
        h = 0
    elif 70 <= height <= 90:
        h = 1
    elif 50 <= height < 70:
        h = 2

    if -30 <= side < -10:
        s = 0
    elif -10 <= side <= 10:
        s = 1
    elif 10 < side <= 30:
        s = 2

    location[h][s] += 1
    total += 1

location

ratio = []

for lst in location:
    add = [num/total for num in lst]
    ratio.append(add)

ratio

plt.figure(figsize=(8, 8))
plt.imshow(ratio, cmap='OrRd')

plt.colorbar()
plt.clim(0, 0.2)
plt.title('ストレートのコース別投球割合')
plt.xticks([0, 1, 2], ['三塁側', '真ん中', '一塁側'])
plt.yticks([0, 1, 2], ['高め', '真ん中', '低め'])
plt.show()

In [ ]:
def create_heatmap(data, title):
    # コース別に集計するためのリスト作成
    location = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
    sum = 0

    for height, side in zip(Fastball['plate_z_cm'], Fastball['plate_x_cm']):
        
        if height < 50 or 110 < height:
            continue
        if side < -30 or 30 < side:
            continue
        
        if 90 < height <= 110:
            h = 0
        elif 70 <= height <= 90:
            h = 1
        elif 50 <= height < 70:
            h = 2
    
        if -30 <= side < -10:
            s = 0
        elif -10 <= side <= 10:
            s = 1
        elif 10 < side <= 30:
            s = 2
    
        location[h][s] += 1
        sum += 1

    # コース別の投球割合を計算
    ratio = []

    for lst in location:
        add = [num/sum for num in lst]
        ratio.append(add)

    # ヒートマップの作成
    plt.figure(figsize=(8, 8))
    plt.imshow(ratio, cmap='OrRd')

    # 見た目の調整
    plt.colorbar()
    plt.clim(0, 0.2)
    plt.title(title)
    plt.xticks([0, 1, 2], ['三塁側', '真ん中', '一塁側'])
    plt.yticks([0, 1, 2], ['高め', '真ん中', '低め'])
    plt.show()

In [ ]:
Fastball_Right = Fastball[Fastball['stand']=='R']
Fastball_Left = Fastball[Fastball['stand']=='L']

create_heatmap(Fastball_Right, 'ストレートのコース別投球割合（対右打者）')
create_heatmap(Fastball_Left, 'ストレートのコース別投球割合（対左打者）')

In [ ]:
df['release_pos_x_cm'] = df['release_pos_x'] * 30.48
df['release_pos_z_cm'] = df['release_pos_z'] * 30.48

Fastball = df[df['pitch_type']=='FF'] #ストレート
Slider = df[df['pitch_type']=='SL'] #スライダー
Sinker = df[df['pitch_type']=='SI'] #シンカー
Splitter = df[df['pitch_type']=='FS'] #スプリット

plt.boxplot([Fastball['release_pos_z_cm'],
             Slider['release_pos_z_cm'],
             Sinker['release_pos_z_cm'],
             Splitter['release_pos_z_cm']],
             labels=['ストレート', 'スライダー', 'シンカー', 'スプリット'])
plt.ylabel('上下リリース位置（cm）')
plt.title('球種別のリリース位置（高さ）')
plt.show()

In [ ]:
plt.boxplot([Fastball['release_pos_z_cm'],
             Slider['release_pos_z_cm'],
             Sinker['release_pos_z_cm'],
             Splitter['release_pos_z_cm']],
             labels=['ストレート', 'スライダー', 'シンカー', 'スプリット'],
             vert=False)
plt.ylabel('上下リリース位置（cm）')
plt.title('球種別のリリース位置（高さ）')
plt.show()